In [1]:
import pandas as pd
import requests
import math
from termcolor import colored as cl

In [4]:
api_key = '6540f7fdcf56d2.23799583'
msft_sentiment = requests.get(f'https://eodhistoricaldata.com/api/tweets-sentiments?s=MSFT.US&from=2022-01-01&to=2023-04-19&api_token={api_key}').json()
msft_hist = requests.get(f'https://eodhistoricaldata.com/api/eod/MSFT.US?from=2022-01-01&to=2023-04-19&period=d&fmt=json&api_token={api_key}').json()

In [5]:
sentiment_df = pd.DataFrame(msft_sentiment['MSFT.US'])[::-1]
hist_df = pd.DataFrame(msft_hist)

msft = sentiment_df.merge(hist_df)
msft = msft.set_index('date')
msft.index = pd.to_datetime(msft.index)
msft.tail()

,count,normalized,open,high,low,close,adjusted_close,volume,warning
date,,,,,,,,,
2023-03-08,486,0.0914,254.04,254.5417,250.81,253.70,252.6118,17340221,NaN
2023-03-09,536,0.1481,255.82,259.5600,251.58,252.32,251.2377,26653391,NaN
2023-03-10,427,0.0766,251.08,252.7900,247.60,248.59,247.5237,28333930,NaN
2023-03-13,584,0.0974,247.40,257.9100,245.73,253.92,252.8308,33339719,NaN
2023-03-14,618,0.1663,256.75,261.0700,255.86,260.79,259.6714,33620289,NaN


In [6]:
in_position = False
investment = 100000
equity = investment

for i in range(len(msft)):
    if msft['normalized'][i] > 0 and in_position == False:
        no_of_shares = math.floor(equity/msft.close[i])
        equity -= no_of_shares * msft.close[i]
        in_position = True
        print(cl('BUY: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at ${msft.close[i]} on {msft.index[i]}')
    elif msft['normalized'][i] < 0 and in_position == True:
        equity += no_of_shares * msft.close[i]
        in_position = False
        print(cl('SELL: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are bought at ${msft.close[i]} on {msft.index[i]}')
if in_position == True:
    equity += no_of_shares * msft.close[i]
    print(cl(f'\nClosing position at {msft.close[i]} on {msft.index[i]}', attrs = ['bold']))
    in_position = False

earning = round(equity - investment, 2)
roi = round(earning / investment * 100, 2)
print(cl(f'EARNING: ${earning} ; ROI: {roi}%', attrs = ['bold']))

BUY:  430 Shares are bought at $232.13 on 2022-10-31 00:00:00
SELL:  430 Shares are bought at $240.35 on 2023-01-17 00:00:00
BUY:  446 Shares are bought at $231.93 on 2023-01-19 00:00:00
SELL:  446 Shares are bought at $263.62 on 2023-02-09 00:00:00
BUY:  447 Shares are bought at $263.1 on 2023-02-10 00:00:00

Closing position at 260.79 on 2023-03-14 00:00:00
EARNING: $16635.77 ; ROI: 16.64%


In [7]:
msft_rsi = requests.get(f'https://eodhistoricaldata.com/api/technical/MSFT.US?fmt=json&from=2022-01-01&to=2023-04-19&function=rsi&period=14&api_token={api_key}').json()
rsi_df = pd.DataFrame(msft_rsi)
rsi_df['date'] = pd.to_datetime(rsi_df.date)

msft = msft.reset_index()
msft = msft.merge(rsi_df)
msft = msft.set_index('date')

in_position = False
investment = 100000
equity = investment

for i in range(len(msft)):
    if msft['normalized'][i] < 0 and msft['rsi'][i] < 40 and in_position == False:
        no_of_shares = math.floor(equity/msft.close[i])
        equity -= no_of_shares * msft.close[i]
        in_position = True
        print(cl('BUY: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at ${msft.close[i]} on {msft.index[i]}')
    elif msft['normalized'][i] > 0 and msft['rsi'][i] > 60 and in_position == True:
        equity += no_of_shares * msft.close[i]
        in_position = False
        print(cl('SELL: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are bought at ${msft.close[i]} on {msft.index[i]}')
if in_position == True:
    equity += no_of_shares * msft.close[i]
    print(cl(f'\nClosing position at {msft.close[i]} on {msft.index[i]}', attrs = ['bold']))
    in_position = False

earning = round(equity - investment, 2)
roi = round(earning / investment * 100, 2)
print(cl(f'\nEARNING: ${earning} ; ROI: {roi}%', attrs = ['bold']))

JSONDecodeError: ignored